In [1]:
!pip install -q arize-phoenix openinference-instrumentation-bedrock

import phoenix as px
import os
import boto3
import time
import json
from phoenix.otel import register

session = px.launch_app()

tracer_provider = register(
    project_name="cerulion-test",
    auto_instrument=True
)

session = boto3.session.Session()
bedrock_agent_client = session.client(
    "bedrock-agent-runtime",
    region_name="us-east-1"
)

AGENT_ID = "<your-agent-id>"
AGENT_ALIAS_ID = "TSTALIASID" # replace TSTALIASID with your agent alias id, otherwise leave it and it will use the latest version

def debug_bedrock_agent(input_text, verbose=True):
    session_id = f"sagemaker_session_{int(time.time())}"
    
    attributes = {
        "inputText": input_text,
        "agentId": AGENT_ID,
        "agentAliasId": AGENT_ALIAS_ID,
        "sessionId": session_id,
        "enableTrace": True,
    }
    
    try:
        response = bedrock_agent_client.invoke_agent(**attributes)
        
        full_response = ""
        step_count = 0
        tool_calls = []
        reasoning_steps = []
        
        for event in response.get("completion", []):
            if "chunk" in event:
                chunk = event["chunk"]
                if "bytes" in chunk:
                    text = chunk["bytes"].decode()
                    full_response += text
            
            if "trace" in event:
                step_count += 1
                trace = event["trace"]
                
                if "orchestrationTrace" in trace:
                    orch = trace["orchestrationTrace"]
                    
                    if "rationale" in orch:
                        reasoning = orch["rationale"]["text"]
                        reasoning_steps.append(reasoning)
                        if verbose:
                            print(f"Step {step_count} Reasoning: {reasoning}")
                    
                    if "invocationInput" in orch:
                        inv_input = orch["invocationInput"]
                        if "actionGroupInvocationInput" in inv_input:
                            action_input = inv_input["actionGroupInvocationInput"]
                            tool_name = action_input.get("actionGroupName", "Unknown")
                            function_name = action_input.get("function", "Unknown")
                            parameters = action_input.get("parameters", {})
                            
                            tool_call = {
                                "tool": tool_name,
                                "function": function_name,
                                "parameters": parameters
                            }
                            tool_calls.append(tool_call)
                            
                            if verbose:
                                print(f"Tool Called: {tool_name} - {function_name}")
                                print(f"Parameters: {parameters}")
                    
                    if "observation" in orch:
                        obs = orch["observation"]
                        if "actionGroupInvocationOutput" in obs:
                            output = obs["actionGroupInvocationOutput"]
                            result_text = output.get("text", "No text output")
                            if verbose:
                                print(f"Tool Result: {result_text}")
        
        if verbose:
            print(f"Total Steps: {step_count}")
            print(f"Tools Used: {len(tool_calls)}")
            print(f"Reasoning Steps: {len(reasoning_steps)}")
            print(f"Final Response: {full_response}")
        
        return {
            "response": full_response,
            "session_id": session_id,
            "steps": step_count,
            "tool_calls": tool_calls,
            "reasoning_steps": reasoning_steps,
            "success": True
        }
        
    except Exception as e:
        if verbose:
            print(f"Error: {str(e)}")
        
        return {
            "response": None,
            "session_id": session_id,
            "error": str(e),
            "success": False
        }

test_input = "What's the weather like in Melbourne today?"
result = debug_bedrock_agent(test_input, verbose=True)

px.active_session().view()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.3 which is incompatible.


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/contextlib.py:142: SAWarning: Skipped unsupported reflection of expression-based index ix_cumulative_llm_token_count_total
  next(self.gen)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/contextlib.py:142: SAWarning: Skipped unsupported reflection of expression-based index ix_latency
  next(self.gen)


🌍 To view the Phoenix app in your browser, visit https://test2.notebook.us-east-1.sagemaker.aws/proxy/6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: oisin-test
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

Total Steps: 4
Tools Used: 0
Reasoning Steps: 0
Final Response: 



{
    "function": "user__askuser",
    "parameters": {
        "question": "Could you please specify which Melbourne you're referring to? (e.g., Melbourne, Australia or Melbourne, Florida, USA)"
